In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

# sys.path.append('./modules')
# from gaussian_mixture_model import GMM

In [7]:
class GMM():
    def __init__(self, num_clusters):
        self.mu = None
        self.Sigma = None # Lambda^{-1}
        self.pi = None
        self.gamma = None
        self.num_clusters = num_clusters # hyperparameter
        self.num_samples = None
        self.dim = None

        return
    
    def fit(self, X, max_iter=1000, threshold=1e-05):
        num_samples = X.shape[0]
        dim = X.shape[1]
        print(dim)

        self.__init_params(dim, num_samples)

        prev = sys.float_info.max / 2
        for iter in range(max_iter):
            # E-step
            self.gamma = self.__calc_gamma(X)

            # M-step
            self.mu = self.__calc_mu(X)
            self.Sigma = self.__calc_Sigma(X)
            self.pi = self.__calc_pi(X)
            
            loss = self.__log_likelihood(X)

            print(f'iter:{iter}\tloss:{loss}')

            if np.abs(prev - loss) < threshold:
                print(f'\t>>>early stop! iter={iter}')
                break
            
            prev = loss
        
        return self

    def __init_params(self, dim, num_samples):
        self.mu = np.random.rand(self.num_clusters, dim)
        self.pi = np.random.rand(self.num_clusters)
        self.gamma = np.random.rand(num_samples, self.num_clusters)
        self.num_samples = num_samples
        self.dim = dim

        self.Sigma = np.zeros((self.num_clusters, self.dim, self.dim))
        for m in range(self.num_clusters):
            while np.linalg.det(self.Sigma[m]) <= 0:
                self.Sigma[m] = self.__gen_rand_sym_mat(self.dim)

        return
    
    def __gen_rand_sym_mat(self, size):
        elements = np.random.rand(size, size)
        matrix = np.triu(elements) + np.triu(elements, 1).T

        return matrix
    
    def __calc_gamma(self, X):
        gamma = np.zeros_like(self.gamma)
        for n in range(self.num_samples):
            p = np.zeros((self.num_clusters))
            sum = 0

            for k in range(self.num_clusters):
                p[k] = self.pi[k] * self.__gauss(X[n].T, self.mu[k].T, self.Sigma[k])
                sum += p[k]
            
            for m in range(self.num_clusters):
                gamma[n][m] = p[m] / sum
        
        return gamma
    
    def __gauss(self, x, mu, Sigma):
        if np.linalg.det(Sigma) == 0:
            print(Sigma)
        Lambda = np.linalg.inv(Sigma)
        c = np.sqrt((2 * np.pi) ** self.dim) * np.sqrt(np.linalg.det(Sigma))
        residue = (x - mu).reshape((-1, 1))
        exponent = (- residue.T @ Lambda @ residue / 2)[0][0]
            
        return np.exp(exponent) / c
    
    def __calc_mu(self, X):
        mu = np.zeros_like(self.mu)

        for m in range(self.num_clusters):
            deno = np.zeros_like(mu[m])
            nume = 0
            for n in range(self.num_samples):
                deno += self.gamma[n][m] * X[n]
                nume += self.gamma[n][m]
            
            mu[m] = deno / nume
        
        return mu
    
    def __calc_Sigma(self, X):
        Sigma = np.zeros_like(self.Sigma)

        for m in range(self.num_clusters):
            deno = np.zeros_like(Sigma[m])
            nume = 0

            for n in range(self.num_samples):
                residue = (X[n] - self.mu[m]).reshape((-1, 1))

                deno += self.gamma[n][m] * residue @ residue.T
                nume += self.gamma[n][m]

            Sigma[m] = deno / nume

        return Sigma
    
    def __calc_pi(self, X):
        return np.sum(self.gamma, axis=0) / self.num_samples
    
    def __log_likelihood(self, X):
        retval = 0
        for n in range(self.num_samples):
            sum = 0
            for k in range(self.num_clusters):
                sum += self.pi[k] * self.__gauss(X[n], self.mu[k], self.Sigma[k])
            
            retval += np.log(sum)
        
        return retval

In [8]:
path = './data/sample_data.csv'
data = pd.read_csv(path).values

X = data[:, :2]  # exclude labels

In [10]:
gmm = GMM(num_clusters=3)
gmm.fit(X)

2
iter:0	loss:-17173.351640959383
iter:1	loss:-17028.379653232372
iter:2	loss:-16958.66094877553
iter:3	loss:-16790.380382415584
iter:4	loss:-16437.368145750446
iter:5	loss:-16202.353231263474
iter:6	loss:-16184.525911124963
iter:7	loss:-16173.192442125146
iter:8	loss:-16162.669230477919
iter:9	loss:-16139.996469492979
iter:10	loss:-16092.846361782804
iter:11	loss:-16007.21240968597
iter:12	loss:-15867.97046454883
iter:13	loss:-15669.478706451251
iter:14	loss:-15427.296850772484
iter:15	loss:-15144.833133420494
iter:16	loss:-14731.323186126661
iter:17	loss:-13978.910338943479
iter:18	loss:-13848.795186017394
iter:19	loss:-13848.795177112463
	>>>early stop! iter=19


In [11]:
for i in range(gmm.num_clusters):
    print(f'cluster {i}')
    print(f'the rate of the size : {gmm.pi[i]} (size : {gmm.pi[i] * X.shape[0]})')
    print(f'the mean vector : {gmm.mu[i]}')
    print(f'the variance-covariance matrix : \n{gmm.Sigma[i]}')
    print()

cluster 0
the rate of the size : 0.4000060086158216 (size : 1200.0180258474647)
the mean vector : [8.02808785 8.9809193 ]
the variance-covariance matrix : 
[[1.91703972 1.03826115]
 [1.03826115 2.0632758 ]]

cluster 1
the rate of the size : 0.2666606580508428 (size : 799.9819741525284)
the mean vector : [-2.02713462 -0.06086033]
the variance-covariance matrix : 
[[1.0650203  2.16653978]
 [2.16653978 9.47978675]]

cluster 2
the rate of the size : 0.3333333333333354 (size : 1000.0000000000063)
the mean vector : [ 9.96778149 -5.96643358]
the variance-covariance matrix : 
[[1.94529402 0.87389255]
 [0.87389255 2.79451354]]

